# Topic Modeling Exercise
- Queen's MMAI 891

In [1]:
import datetime
print(datetime.datetime.now())

2019-01-16 10:25:49.732236


In [2]:
import pandas as pd
import os

# Make sure the text doesn't get truncated when printed
pd.set_option('display.max_colwidth', -1)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Read in the Data

In [3]:
# Change these paths to where you downloaded the data
in_dir = "C:/Users/st50/Documents/sandbox/data"
out_dir = "C:/Users/st50/Documents/sandbox/out"

df = pd.read_csv(os.path.join(in_dir, "election-tweets-2016.csv"))

list(df)
df.info()
df.shape
df.head()
df.tail()

['IsRetweet',
 'Time',
 'Language',
 'RetweetCount',
 'FavoriteCount',
 'Longitude',
 'Latitude',
 'Author',
 'SourceURL',
 'Content',
 'OriginalAuthor',
 'Place']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6444 entries, 0 to 6443
Data columns (total 12 columns):
IsRetweet         6444 non-null bool
Time              6444 non-null object
Language          6444 non-null object
RetweetCount      6444 non-null float64
FavoriteCount     6444 non-null float64
Longitude         12 non-null float64
Latitude          12 non-null float64
Author            6444 non-null object
SourceURL         6444 non-null object
Content           6444 non-null object
OriginalAuthor    722 non-null object
Place             204 non-null object
dtypes: bool(1), float64(4), object(7)
memory usage: 560.2+ KB


(6444, 12)

,IsRetweet,Time,Language,RetweetCount,FavoriteCount,Longitude,Latitude,Author,SourceURL,Content,OriginalAuthor,Place
0,False,2016-09-28 00:22:34,en,218.0,651.0,NaN,NaN,HillaryClinton,https://studio.twitter.com,The question in this election: Who can put the plans into action that will make your life better? https://t.co/XreEY9OicG,NaN,NaN
1,True,2016-09-27 23:45:00,en,2445.0,5308.0,NaN,NaN,HillaryClinton,http://twitter.com,"Last night, Donald Trump said not paying taxes was ""smart."" You know what I call it? Unpatriotic. https://t.co/t0xmBfj7zF",timkaine,NaN
2,True,2016-09-27 23:26:40,en,7834.0,27234.0,NaN,NaN,HillaryClinton,https://about.twitter.com/products/tweetdeck,Couldn't be more proud of @HillaryClinton. Her vision and command during last night's debate showed that she's ready to be our next @POTUS.,POTUS,NaN
3,False,2016-09-27 23:08:41,en,916.0,2542.0,NaN,NaN,HillaryClinton,https://studio.twitter.com,"If we stand together, there's nothing we can't do. Make sure you're ready to vote: https://t.co/tTgeqxNqYm https://t.co/Q3Ymbb7UNy",NaN,NaN
4,False,2016-09-27 22:30:27,en,859.0,2882.0,NaN,NaN,HillaryClinton,https://about.twitter.com/products/tweetdeck,Both candidates were asked about how they'd confront racial injustice. Only one had a real answer. https://t.co/sjnEokckis,NaN,NaN


,IsRetweet,Time,Language,RetweetCount,FavoriteCount,Longitude,Latitude,Author,SourceURL,Content,OriginalAuthor,Place
6439,False,2016-01-05 03:47:14,en,1110.0,4024.0,NaN,NaN,realDonaldTrump,http://twitter.com/download/android,"""@lilredfrmkokomo: @realDonaldTrump My Facebook Groups are all voting TRUMP /4000 people! !!"" Great!",NaN,NaN
6440,False,2016-01-05 03:44:17,en,855.0,3181.0,NaN,NaN,realDonaldTrump,http://twitter.com/download/android,"""@marybnall01: @realDonaldTrump watched lowell mass speech. Awesome. Great crowd. Make America Great Again!!!!!!""",NaN,NaN
6441,False,2016-01-05 03:42:10,en,2315.0,5992.0,NaN,NaN,realDonaldTrump,http://twitter.com/download/android,"""@ghosthunter_lol: Iowa key endorsement for @realDonaldTrump Can't wait for the Iowa caucus in 4 weeks! #Trump2016 https://t.co/JBfyFrZfFb""",NaN,NaN
6442,False,2016-01-05 03:39:11,en,1054.0,3258.0,NaN,NaN,realDonaldTrump,http://twitter.com/download/android,"""@iLoveiDevices: @EdwinRo47796972 @happyjack225 @FoxNews @krauthammer Minimizing dependency on China is crucial.Only Trump talks about that",NaN,NaN
6443,False,2016-01-05 03:36:53,en,748.0,2658.0,NaN,NaN,realDonaldTrump,http://twitter.com/download/android,"""@SalRiccobono: @realDonaldTrump @troyconway Donald get big business back and# MAKE AMERICA GREAT AGAIN FOR 2016""",NaN,NaN


# Text Preprocessing

Here is where the preprocess magic happens. In the below, I've written a function called `preprocess` that does a bunch of standard steps: lower casing, removing puncuation, etc. You can comment some of these steps out, or add more of your own. Up to you!

In [4]:
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import unidecode
import re

stop_words = set(stopwords.words('english') + stopwords.words('spanish'))

lemmer = WordNetLemmatizer()

def preprocess(x):
    # Lower case
    x = x.lower()
    
    # Remove punctuation
    x = re.sub(r'[^\w\s]', '', x)
    
    # Remove non-unicode
    x = unidecode.unidecode(x)
    
    # Remove numbers
    x = re.sub(r'\d+', '', x)
    
    # Remove stopwords and lemmatize
    x = [lemmer.lemmatize(w) for w in x.split() if w not in stop_words]
    return ' '.join(x) 

df['Content_Clean'] = df['Content'].apply(preprocess)

Now, let's look at some of the preprocessed documents.

In [5]:
df.head()
df.tail()

,IsRetweet,Time,Language,RetweetCount,FavoriteCount,Longitude,Latitude,Author,SourceURL,Content,OriginalAuthor,Place,Content_Clean
0,False,2016-09-28 00:22:34,en,218.0,651.0,NaN,NaN,HillaryClinton,https://studio.twitter.com,The question in this election: Who can put the plans into action that will make your life better? https://t.co/XreEY9OicG,NaN,NaN,question election put plan action make life better httpstcoxreeyoicg
1,True,2016-09-27 23:45:00,en,2445.0,5308.0,NaN,NaN,HillaryClinton,http://twitter.com,"Last night, Donald Trump said not paying taxes was ""smart."" You know what I call it? Unpatriotic. https://t.co/t0xmBfj7zF",timkaine,NaN,last night donald trump said paying tax smart know call unpatriotic httpstcotxmbfjzf
2,True,2016-09-27 23:26:40,en,7834.0,27234.0,NaN,NaN,HillaryClinton,https://about.twitter.com/products/tweetdeck,Couldn't be more proud of @HillaryClinton. Her vision and command during last night's debate showed that she's ready to be our next @POTUS.,POTUS,NaN,couldnt proud hillaryclinton vision command last night debate showed shes ready next potus
3,False,2016-09-27 23:08:41,en,916.0,2542.0,NaN,NaN,HillaryClinton,https://studio.twitter.com,"If we stand together, there's nothing we can't do. Make sure you're ready to vote: https://t.co/tTgeqxNqYm https://t.co/Q3Ymbb7UNy",NaN,NaN,stand together there nothing cant make sure youre ready vote httpstcottgeqxnqym httpstcoqymbbuny
4,False,2016-09-27 22:30:27,en,859.0,2882.0,NaN,NaN,HillaryClinton,https://about.twitter.com/products/tweetdeck,Both candidates were asked about how they'd confront racial injustice. Only one had a real answer. https://t.co/sjnEokckis,NaN,NaN,candidate asked theyd confront racial injustice one real answer httpstcosjneokckis


,IsRetweet,Time,Language,RetweetCount,FavoriteCount,Longitude,Latitude,Author,SourceURL,Content,OriginalAuthor,Place,Content_Clean
6439,False,2016-01-05 03:47:14,en,1110.0,4024.0,NaN,NaN,realDonaldTrump,http://twitter.com/download/android,"""@lilredfrmkokomo: @realDonaldTrump My Facebook Groups are all voting TRUMP /4000 people! !!"" Great!",NaN,NaN,lilredfrmkokomo realdonaldtrump facebook group voting trump people great
6440,False,2016-01-05 03:44:17,en,855.0,3181.0,NaN,NaN,realDonaldTrump,http://twitter.com/download/android,"""@marybnall01: @realDonaldTrump watched lowell mass speech. Awesome. Great crowd. Make America Great Again!!!!!!""",NaN,NaN,marybnall realdonaldtrump watched lowell mass speech awesome great crowd make america great
6441,False,2016-01-05 03:42:10,en,2315.0,5992.0,NaN,NaN,realDonaldTrump,http://twitter.com/download/android,"""@ghosthunter_lol: Iowa key endorsement for @realDonaldTrump Can't wait for the Iowa caucus in 4 weeks! #Trump2016 https://t.co/JBfyFrZfFb""",NaN,NaN,ghosthunter_lol iowa key endorsement realdonaldtrump cant wait iowa caucus week trump httpstcojbfyfrzffb
6442,False,2016-01-05 03:39:11,en,1054.0,3258.0,NaN,NaN,realDonaldTrump,http://twitter.com/download/android,"""@iLoveiDevices: @EdwinRo47796972 @happyjack225 @FoxNews @krauthammer Minimizing dependency on China is crucial.Only Trump talks about that",NaN,NaN,iloveidevices edwinro happyjack foxnews krauthammer minimizing dependency china crucialonly trump talk
6443,False,2016-01-05 03:36:53,en,748.0,2658.0,NaN,NaN,realDonaldTrump,http://twitter.com/download/android,"""@SalRiccobono: @realDonaldTrump @troyconway Donald get big business back and# MAKE AMERICA GREAT AGAIN FOR 2016""",NaN,NaN,salriccobono realdonaldtrump troyconway donald get big business back make america great


## Topic Modeling with Sci-kit Learn

First, we need to vectorize the text.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# The number of "features" (i.e., tokens) we want to keep in the BOW.
no_features = 1000

vectorizer = CountVectorizer(max_df=0.99, min_df=0.0001, 
                                max_features=no_features, ngram_range=[1,3])
%time dtm = vectorizer.fit_transform(df['Content_Clean'])
print(dtm.shape)

Wall time: 445 ms
(6444, 1000)


And just for fun, let's look at the top 30 tokens

In [7]:
from yellowbrick.text import FreqDistVisualizer

feature_names = vectorizer.get_feature_names()
visualizer = FreqDistVisualizer(features=feature_names, n=30)
visualizer.fit(dtm)
visualizer.poof()

FrequencyVisualizer(ax=<matplotlib.axes._subplots.AxesSubplot object at 0x0000011E2A120FD0>,
          color=None,
          features=['abc', 'able', 'access', 'account', 'across', 'act', 'action', 'actually', 'ad', 'add', 'address', 'afford', 'affordable', 'african', 'african american', 'ago', 'agree', 'ahead', 'air', 'allowed', 'ally', 'almost', 'alone', 'along', 'already', 'also', 'always', 'amazing', 'amendment', 'ame...', 'year', 'year ago', 'yes', 'yesterday', 'yet', 'york', 'young', 'youre', 'youve', 'zero', 'zika'],
          n=None, orient='h')

<Figure size 800x550 with 1 Axes>

Now, we run LDA.

In [8]:
from sklearn.decomposition import LatentDirichletAllocation

alpha=None
beta=None

lda_model = LatentDirichletAllocation(n_components=25,
                                      doc_topic_prior=alpha,
                                      topic_word_prior=beta,
                                      max_iter=200, 
                                      learning_method='online', 
                                      random_state=123,
                                      n_jobs=2,
                                      verbose=0)
%time lda_output = lda_model.fit(dtm)

# Log Likelihood: Higher the better
lda_model.score(dtm)

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
lda_model.perplexity(dtm)

# Theta = document-topic matrix
# Beta = components_ = topic-term matrix
theta = pd.DataFrame(lda_model.transform(dtm))
beta = pd.DataFrame(lda_model.components_)

Wall time: 2min 32s


-322043.1504325381

828.3630019376671

# Look at the Topics

Let's inspect the topics. Let's figure out the top words. Also, let's compute some simple metrics on each topic, like how many documents contain it (_support_) and what the total size of the topic is (_weight_).

In [9]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 0)

# How many words to display for each topic
no_top_words = 10
weight = theta.sum(axis=0)

# Number of documents that contain a topic more than 50%
support50 = (theta > 0.5).sum(axis=0)

# Number of documents that contain a document more than 10%
support10 = (theta > 0.1).sum(axis=0)
termss = list()
for topic_id, topic in enumerate(lda_model.components_):
    terms = " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
    termss.append(terms)
topic_summary = pd.DataFrame({'TopicID': range(0, len(termss)), "Support50": support50, "Support10": support10, "Weight": weight, "Terms": termss})

# Display the topics; sort by Weight
topic_summary.sort_values('Weight', ascending=False)

,TopicID,Support50,Support10,Weight,Terms
24,24,252,1381,449.149186,thank trump amp makeamericagreatagain today poll new fight gun national
3,3,156,1431,439.429290,trump donald donald trump woman cant president say isi first everything
4,4,75,1249,340.968058,hillary clinton hillary clinton campaign world president stand youre crooked hillary clinton proud
7,7,153,1044,332.284661,big tonight speech pm join back enjoy crowd tomorrow iowa
12,12,36,1283,326.863126,hillary crooked crooked hillary know one best ad bad could word
22,22,113,936,282.566833,great america make make america america great make america great carolina americafirst safe lead
17,17,33,1126,281.972704,people like republican see trump party primary help american house
15,15,49,991,268.108492,hillary watch take plan life live stop policy put week
21,21,35,1006,262.812134,job good look tax trump even time done senator well
0,0,30,989,260.076383,country love show great bernie wow nothing supporter sad sander


# Show some Documents that Match a Given Topic

In [10]:
# Make sure the text doesn't get truncated when printed
pd.set_option('display.max_colwidth', -1)

# Which topic are you interested in?
topic_id = 23

# Find the 5 documents with the largets membership (i.e., theta) for this topic
Memberships = theta.iloc[:, topic_id].nlargest(5)

Memberships

# Display those documents (and getting rid of some of the columns)
df[['Time', 'RetweetCount', 'FavoriteCount', 'Author', 'Content', 'Content_Clean']].iloc[Memberships.index]

5137    0.630000
5391    0.608000
5974    0.608000
368     0.577143
5377    0.520000
Name: 23, dtype: float64

,Time,RetweetCount,FavoriteCount,Author,Content,Content_Clean
5137,2016-03-24 16:02:06,3998.0,10462.0,realDonaldTrump,Endorsements for Lyin' Ted Cruz- https://t.co/c3QEbexOwT,endorsement lyin ted cruz httpstcocqebexowt
5391,2016-03-08 02:47:19,4668.0,8877.0,realDonaldTrump,Ted Cruz Was For Welcoming Syrian Refugees Before He Was Against It https://t.co/zw4o1DJgfB,ted cruz welcoming syrian refugee httpstcozwodjgfb
5974,2016-02-03 20:44:28,3164.0,9177.0,realDonaldTrump,"Dr. Ben Carson blasted Ted Cruz for ""deceit and dirty tricks and lies.""",dr ben carson blasted ted cruz deceit dirty trick lie
368,2016-09-19 00:24:51,13167.0,25139.0,HillaryClinton,"""Progress is on the ballot. Tolerance is on the ballot. Democracy is on the ballot. Justice is on the ballot."" https://t.co/Mmyqestken",progress ballot tolerance ballot democracy ballot justice ballot httpstcommyqestken
5377,2016-03-08 17:52:32,2988.0,7064.0,realDonaldTrump,Hawaii: https://t.co/MnIlk2l9hP Idaho: https://t.co/7y5RxLpZRQ Mississippi: https://t.co/n43cPeJIqa Michigan: https://t.co/GL5JiZbqIc,hawaii httpstcomnilklhp idaho httpstcoyrxlpzrq mississippi httpstconcpejiqa michigan httpstcogljizbqic


# Visualize Topics with LDAVis

In [11]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
% time pyLDAvis.sklearn.prepare(lda_model, dtm, vectorizer, mds="tsne")

C:\Users\st50\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


Wall time: 16.1 s


PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
3      181.499115  122.782234  1       1        6.796536
24    -84.350151  -215.350235  2       1        5.871369
12    -212.237686 -62.397106   3       1        5.524952
4      67.698402  -133.051926  4       1        5.349907
7      114.698624 -52.282772   5       1        4.726983
22    -61.894375   186.094528  6       1        4.725679
17    -185.988510 -173.327072  7       1        4.552740
21     50.927586   201.200302  8       1        4.391946
15    -164.535461  145.969254  9       1        4.265347
0      19.289850  -227.996231  10      1        4.217750
16    -7.856592    107.049385  11      1        4.087018
19    -31.688801   22.630465   12      1        4.020157
10    -207.935226  45.519737   13      1        3.938316
18    -117.302017 -109.530144  14      1        3.915960
11     125.329575 -208.970459  15      1        3.895402
14    -52.944221  -55.846664   16      1        3.892730
2     -127.100998 -9.279407    17      1        3.358357
1      139.484833  33.429356   18      1        3.313421
20     82.678520   110.767433  19      1        3.243252
13     218.920349 -17.084879   20      1        3.180174
6     -98.737434   78.358269   21      1        2.951196
8     -23.626778  -139.064117  22      1        2.856981
5      51.126724   24.823021   23      1        2.758866
9      181.626617 -119.061478  24      1        2.277877
23     28.100246  -56.211651   25      1        1.887087, topic_info=     Category         Freq   ...     loglift  logprob
term                         ...                     
882   Default  1569.000000   ...     30.0000  30.0000
372   Default  1068.000000   ...     29.0000  29.0000
692   Default  435.000000    ...     28.0000  28.0000
841   Default  522.000000    ...     27.0000  27.0000
333   Default  524.000000    ...     26.0000  26.0000
626   Default  410.000000    ...     25.0000  25.0000
29    Default  407.000000    ...     24.0000  24.0000
202   Default  453.000000    ...     23.0000  23.0000
203   Default  417.000000    ...     22.0000  22.0000
501   Default  378.000000    ...     21.0000  21.0000
130   Default  318.000000    ...     20.0000  20.0000
311   Default  283.000000    ...     19.0000  19.0000
35    Default  324.000000    ...     18.0000  18.0000
168   Default  225.000000    ...     17.0000  17.0000
983   Default  225.000000    ...     16.0000  16.0000
923   Default  253.000000    ...     15.0000  15.0000
578   Default  274.000000    ...     14.0000  14.0000
373   Default  236.000000    ...     13.0000  13.0000
721   Default  177.000000    ...     12.0000  12.0000
935   Default  207.000000    ...     11.0000  11.0000
506   Default  235.000000    ...     10.0000  10.0000
265   Default  211.000000    ...     9.0000   9.0000 
155   Default  203.000000    ...     8.0000   8.0000 
968   Default  237.000000    ...     7.0000   7.0000 
33    Default  287.000000    ...     6.0000   6.0000 
830   Default  156.000000    ...     5.0000   5.0000 
573   Default  190.000000    ...     4.0000   4.0000 
425   Default  217.000000    ...     3.0000   3.0000 
510   Default  169.000000    ...     2.0000   2.0000 
655   Default  371.000000    ...     1.0000   1.0000 
...       ...         ...    ...        ...      ... 
56    Topic25  18.543207     ...     3.9204  -3.8872 
98    Topic25  18.156668     ...     3.9193  -3.9083 
678   Topic25  17.258082     ...     3.9167  -3.9590 
878   Topic25  16.482971     ...     3.9143  -4.0050 
145   Topic25  16.335707     ...     3.9138  -4.0139 
269   Topic25  15.469740     ...     3.9108  -4.0684 
571   Topic25  13.835660     ...     3.9040  -4.1800 
533   Topic25  16.304019     ...     3.7026  -4.0159 
929   Topic25  0.044307      ...    -1.6708  -9.9239 
318   Topic25  0.044307      ...    -1.6812  -9.9239 
376   Topic25  0.044307      ...    -1.6971  -9.9239 
969   Topic25  0.044307      ...    -1.706